Visualize the correlators in th UV coordinates. There is the option to look for U-U correlations (outgoing null direction, keep V fixed), and for V-V correlators (keep u fixed). You need to first run the postprocessing tool "calculate_correlators_UV.jl" for the same NU, NV. By default the V-V correlators are commented out.

You need a semiclassical simulation (with or without backreaction) and store "data" and "quantum", at the same timesteps.

In [ ]:
# load packages
using HDF5
using LaTeXStrings
using Plots ; pythonplot()
using DelimitedFiles
using SpheriCo

In [ ]:
# give the directory where the data from all the runs are saved
dir = "../examples/quantum_runs/"
par = "a0.75_b0.0_c1.0_rmax30.0_tmax8.0_cfl0.0625_sigma0.02_overMp2_1.0_reg_true_backreact_false_mPV1.0_dk_denom_30_kmax20.0_lmax60.0"
your_dir = dir*par

# check if the UV domain (a bit further down) looks ok with that many points
NU = 400 # index i, labels rows, where mat[i,j] is a matrix
NV = 400; # index j, labels columns, where mat[i,j] is a matrix

In [ ]:
# set number of radial grid points
D = 3
Nr = 128*2^D + 3 # the overal course graining

# make directory to save figures
out_dir = "./correlators_UV/"*par*"/resol_D$(D)"
if ispath(out_dir)==false
    mkpath(out_dir)
end

# load the r grid
r = h5read(your_dir*"/data_$(Nr)/r.h5","r")
dr = r[2] - r[1]

# list all available iterations (and corresponding files)
(its, all_filenames) = list_h5_files(your_dir*"/data_$(Nr)", prefix="data_");

# print list of data
println("length(its) = ",length(its))
println("its: ",its)

Store various quantities in (t,r) coordinates. Needed to transfor to double null (U,V) coordinates.

In [ ]:
ti_max = 69#length(its)

U_tr = zeros(ti_max, length(r));
V_tr = zeros(ti_max, length(r));
f_tr = zeros(ti_max, length(r));
g_tr = zeros(ti_max, length(r));

Φ_tr = zeros(ti_max, length(r));
α_tr  = zeros(ti_max, length(r));
A_tr  = zeros(ti_max, length(r));

for i in 1:ti_max
    it = its[i]
    it_str  = lpad(it, 4, "0")

    #v_classic_labels = ["Φ", "Π", "Ψ", "A", "B", "DB", "Utld", "K", "KB", "λ", "α", "Dα", "Θ", "Zr", "f", "g", "U", "V"]
    U   =  h5read(your_dir*"/data_$(Nr)/data_$(it_str).h5","v")[:,17]
    V   =  h5read(your_dir*"/data_$(Nr)/data_$(it_str).h5","v")[:,18]
    f   =  h5read(your_dir*"/data_$(Nr)/data_$(it_str).h5","v")[:,15]
    g   =  h5read(your_dir*"/data_$(Nr)/data_$(it_str).h5","v")[:,16]
    Φ   =  h5read(your_dir*"/data_$(Nr)/data_$(it_str).h5","v")[:,1]
    α   =  h5read(your_dir*"/data_$(Nr)/data_$(it_str).h5","v")[:,11]
    A   =  h5read(your_dir*"/data_$(Nr)/data_$(it_str).h5","v")[:,4]
    
    U_tr[i,:] = U
    V_tr[i,:] = V
    f_tr[i,:] = f
    g_tr[i,:] = g
    Φ_tr[i,:] = Φ 
    α_tr[i,:]  = α
    A_tr[i,:]  = A

end

dtdU_tr = 0.5./(α_tr.*f_tr);
drdU_tr = - 0.5./sqrt.(A_tr.*f_tr); # sometimes the quantity under sqrt is negative (problem).
# You can check where, uncommenting the cell below.

# set manually: 64 is how often (in iterations) we saved data, and 16 is from the CFL
dt = 64*dr/16
tlist = zeros(ti_max) # time list
for i in 1:ti_max-1
    tlist[i+1] = tlist[i]+dt
end


Get the apparent horizon position in (t,r):

The algorithm to go form (t,r) to (U,V) involves the two cells below. First, setting up some necessary quantities, and initiating some arrays to save data in (U,V) coordinates.

In [ ]:
#take min max of U(t,r), and V(t,r)
Umin = minimum(U_tr)
Umax = maximum(U_tr)
Vmin = minimum(V_tr)
Vmax = maximum(V_tr)

U_axis = zeros(NU)
for i in 1:NU
    U_axis[i] = Umin + (i-1)*(Umax - Umin)/(NU-1)
end

V_axis = zeros(NV)
for j in 1:NV
    V_axis[j] = Vmin + (j-1)*(Vmax - Vmin)/(NV-1)
end

dom      = NaN* zeros(NU,NV)
Φ_UV     = NaN* zeros(NU,NV)

Umin = minimum(U_tr)
Umax = maximum(U_tr)
Vmin = minimum(V_tr)
Vmax = maximum(V_tr)

dU = (Umax - Umin)/(NU-1)
dV = (Vmax - Vmin)/(NV-1);

Check if the UV cell and the tr cell have overlap, and if yes, copy the tr data to UV. 
(more details on thi algorithm, in the paper)

In [ ]:
for i in 1:ti_max#length(tlist)
    for j in 1:length(r)
        Utr     = U_tr[i,j]
        Vtr     = V_tr[i,j]
        Φtr     = Φ_tr[i,j]
        
        idown  = Int( round( 1 + (NU - 1)*(Utr - Umin)/(Umax - Umin) ) )
        jleft  = Int( round( 1 + (NV - 1)*(Vtr - Vmin)/(Vmax - Vmin) ) )

        if jleft < NV
            jright = jleft + 1
        else
            jright = jleft
        end
        if idown < NU
            iup =  idown + 1
        else
            iup =  idown
        end

        dom[idown, jleft]  = 1
        dom[iup, jleft]    = 1
        dom[idown, jright] = 1
        dom[iup, jright]   = 1

        Φ_UV[idown, jleft]  = Φtr
        Φ_UV[iup, jleft]    = Φtr
        Φ_UV[idown, jright] = Φtr
        Φ_UV[iup, jright]   = Φtr

    end
end


The position of the apparent horizon, in UV.

In [ ]:
contourf(V_axis, U_axis, dom,
    xlabel = "V",
    ylabel = "U",
    color=:thermal,
    wsize = (500,500),title = "domain", frame=true)


Store the apparent horizon in U,V, as lists. Useful for later.

Plot the scalar field that drives the collapse, in (U,V). It helps to understand what type of correlators we are looking for. In the example below, the vertical black line is a possible slice in which we inspect the U-U correlator.

In [ ]:
# set manually the NU, NV reange you want to plot, below
ui = 200
uj = 400
vi = 1
vj = 200

vslice = 10
println(V_axis[vslice])

p = contourf(V_axis[vi:vj], U_axis[ui:uj], (Φ_UV[ui:uj,vi:vj]),
    xlabel = "V",
    ylabel = "U",
    color=:thermal,
    wsize = (700,700),
    title = L"\Phi(U,V)",
    frame =true)

plot!([V_axis[vslice]], seriestype="vline", label="", linewidth=1.5, color = "green")
# uncomment if you want to save fig
#savefig(p, out_dir*"/Phi_UV.pdf")

U-U correlators:

In [ ]:
uu_crlt = h5read(your_dir*"/data_$(Nr)/UU_crlt_NU$(NU)_vi$(vslice).h5","crlt");

#p1
ui = 200
uj = 400
vi = 1
vj = 200

contourf(V_axis[vi:vj], U_axis[ui:uj], (Φ_UV[ui:uj,vi:vj]),
    xlabel = "V",
    ylabel = "U",
    color=:thermal,
    wsize = (700,700),
    title = L"\Phi(U,V)",
    frame =true,
    legendfontsize=8,
    legend=:bottomleft)

p1 = plot!([V_axis[vslice]], seriestype="vline", label="U-U correlator", linewidth=1.5, color = "green",legend=:bottomleft)

# p2
i = 320
j = 350
contourf(U_axis[i:j], U_axis[i:j], real.((uu_crlt)[i:j,i:j]),
    title="U-U correlator @ V = $(V_axis[vslice])",
    xlabel = "U",
    ylabel = "U",
    color=:thermal,
    frame=true,
    legendfontsize=8,
legend=:bottomright
)

ui = 330
plot!([U_axis[ui]], seriestype="hline", label="U = $(U_axis[ui])", linewidth=1.5, color = "green")
p2 = plot!([U_axis[ui]], seriestype="vline", label="", linewidth=1.5, color = "green")

# p3
plot(U_axis[i:j],  real.(uu_crlt[i:j,ui]),
    xlabel = "U",
    frame=true,
    linewidth = 2, legendfontsize=8,
    label="U-U correlation",
xlim = (U_axis[i],U_axis[j]))

p3 = plot!([U_axis[ui]], seriestype="vline", label="U = $(U_axis[ui])", linewidth=1.5, color = "green", legend=:topleft)

plt = plot(p1, p2, p3, layout = grid(1, 3), size = (1200, 400))

#savefig(plt, out_dir*"/UU_crlt_NU$(NU)_vi$(vslice)_ui$(ui).pdf")

In [ ]:
uu_crlt_10 = h5read(your_dir*"/data_$(Nr)/UU_crlt_NU$(NU)_vi10.h5","crlt");

uu_crlt_60 = h5read(your_dir*"/data_$(Nr)/UU_crlt_NU$(NU)_vi60.h5","crlt");

uu_crlt_85 = h5read(your_dir*"/data_$(Nr)/UU_crlt_NU$(NU)_vi85.h5","crlt");

uu_crlt_110 = h5read(your_dir*"/data_$(Nr)/UU_crlt_NU$(NU)_vi110.h5","crlt");


In [ ]:
#p11
ui = 200
uj = 400
vi = 1
vj = 200

contourf(V_axis[vi:vj], U_axis[ui:uj], (Φ_UV[ui:uj,vi:vj]),
    xlabel = "V",
    ylabel = "U",
    color=:thermal,
    wsize = (700,700),
    title = L"\Phi(U,V)",
    frame =true,
    legendfontsize=8,
    legend=:bottomleft)

p11 = plot!([V_axis[10]], seriestype="vline", label="V = $(V_axis[10])", linewidth=1.5, color = "green",legend=:bottomleft)

# p12
i = 320
j = 350
contourf(U_axis[i:j], U_axis[i:j], real.((uu_crlt_10)[i:j,i:j]),
    title="U-U correlator",
    xlabel = "U",
    ylabel = "U",
    color=:thermal,
    frame=true,
    legendfontsize=8,
legend=:bottomright
)

ui = 330
plot!([U_axis[ui]], seriestype="hline", label="U = $(U_axis[ui])", linewidth=1.5, color = "green")
p12 = plot!([U_axis[ui]], seriestype="vline", label="", linewidth=1.5, color = "green")

# p13
plot(U_axis[i:j],  real.(uu_crlt_10[i:j,ui]),
    xlabel = "U",
    frame=true,
    linewidth = 2, legendfontsize=8,
    label="",
    title = "U-U correlator",
xlim = (U_axis[i],U_axis[j]))

p13 = plot!([U_axis[ui]], seriestype="vline", label="U = $(U_axis[ui])", linewidth=1.5, color = "green", legend=:topleft)

#####################################################

#p21
ui = 200
uj = 400
vi = 1
vj = 200
contourf(V_axis[vi:vj], U_axis[ui:uj], (Φ_UV[ui:uj,vi:vj]),
    xlabel = "V",
    ylabel = "U",
    color=:thermal,
    wsize = (700,700),
    frame =true,
    legendfontsize=8,
    legend=:bottomleft)

p21 = plot!([V_axis[60]], seriestype="vline", label="V = $(V_axis[60])", linewidth=1.5, color = "green",legend=:bottomleft)

# p22
i = 200
j = 400
contourf(U_axis[i:j], U_axis[i:j], real.((uu_crlt_60)[i:j,i:j]),
    xlabel = "U",
    ylabel = "U",
    color=:thermal,
    frame=true,
    legendfontsize=8,
legend=:bottomright
)

ui = 320
plot!([U_axis[ui]], seriestype="hline", label="U = $(U_axis[ui])", linewidth=1.5, color = "green")
p22 = plot!([U_axis[ui]], seriestype="vline", label="", linewidth=1.5, color = "green")

# p23
plot(U_axis[i:j],  real.(uu_crlt_60[i:j,ui]),
    xlabel = "U",
    frame=true,
    linewidth = 2, legendfontsize=8,
    label="",
xlim = (U_axis[i],U_axis[j]))

p23 = plot!([U_axis[ui]], seriestype="vline", label="U = $(U_axis[ui])", linewidth=1.5, color = "green", legend=:bottomleft)

#####################################################

#p31
ui = 200
uj = 400
vi = 1
vj = 200
contourf(V_axis[vi:vj], U_axis[ui:uj], (Φ_UV[ui:uj,vi:vj]),
    xlabel = "V",
    ylabel = "U",
    color=:thermal,
    wsize = (700,700),
    frame =true,
    legendfontsize=8,
    legend=:bottomleft)

p31 = plot!([V_axis[85]], seriestype="vline", label="V = $(V_axis[85])", linewidth=1.5, color = "green",legend=:bottomleft)

# p32
contourf(U_axis[i:j], U_axis[i:j], real.((uu_crlt_85)[i:j,i:j]),
    xlabel = "U",
    ylabel = "U",
    color=:thermal,
    frame=true,
    legendfontsize=8,
legend=:bottomright
)

ui = 320
plot!([U_axis[ui]], seriestype="hline", label="U = $(U_axis[ui])", linewidth=1.5, color = "green")
p32 = plot!([U_axis[ui]], seriestype="vline", label="", linewidth=1.5, color = "green")

# p33
plot(U_axis[i:j],  real.(uu_crlt_85[i:j,ui]),
    xlabel = "U",
    frame=true,
    linewidth = 2, legendfontsize=8,
    label="",
xlim = (U_axis[i],U_axis[j]))

p33 = plot!([U_axis[ui]], seriestype="vline", label="U = $(U_axis[ui])", linewidth=1.5, color = "green", legend=:bottomleft)

#####################################################

#p41
ui = 200
uj = 400
vi = 1
vj = 200
contourf(V_axis[vi:vj], U_axis[ui:uj], (Φ_UV[ui:uj,vi:vj]),
    xlabel = "V",
    ylabel = "U",
    color=:thermal,
    wsize = (700,700),
    frame =true,
    legendfontsize=8,
    legend=:bottomleft)

p41 = plot!([V_axis[110]], seriestype="vline", label="V = $(V_axis[110])", linewidth=1.5, color = "green",legend=:bottomleft)

# p42
contourf(U_axis[i:j], U_axis[i:j], real.((uu_crlt_110)[i:j,i:j]),
    xlabel = "U",
    ylabel = "U",
    color=:thermal,
    frame=true,
    legendfontsize=8,
legend=:bottomright
)

ui = 320
plot!([U_axis[ui]], seriestype="hline", label="U = $(U_axis[ui])", linewidth=1.5, color = "green")
p42 = plot!([U_axis[ui]], seriestype="vline", label="", linewidth=1.5, color = "green")

# p43
plot(U_axis[i:j],  real.(uu_crlt_110[i:j,ui]),
    xlabel = "U",
    frame=true,
    linewidth = 2, legendfontsize=8,
    label="",
xlim = (U_axis[i],U_axis[j]))

p43 = plot!([U_axis[ui]], seriestype="vline", label="U = $(U_axis[ui])", linewidth=1.5, color = "green", legend=:topleft)


#####################################################

plt = plot(
    p11, p12, p13,
    p21, p22, p23,
    p31, p32, p33,
    p41, p42, p43,
    layout = grid(4, 3),
    size = (1200, 4*300))

savefig(plt, out_dir*"/UU_crlt_NU$(NU)_many_slices.pdf")

In [ ]:
# U slice for VV correlators

# set manually the NU, NV reange you want to plot, below
ui = 200
uj = 400
vi = 1
vj = 200

uslice = 280
println(V_axis[uslice])

p = contourf(V_axis[vi:vj], U_axis[ui:uj], (Φ_UV[ui:uj,vi:vj]),
    xlabel = "V",
    ylabel = "U",
    color=:thermal,
    wsize = (700,700),
    title = L"\Phi(U,V)",
    frame =true)

plot!([U_axis[uslice]], seriestype="hline", label="", linewidth=1.5, color = "green")
# uncomment if you want to save fig
#savefig(p, out_dir*"/Phi_UV.pdf")

In [ ]:
uslice = 300
vv_crlt = h5read(your_dir*"/data_$(Nr)/VV_crlt_NV$(NV)_ui$(uslice).h5","crlt");

#p1
ui = 200
uj = 400
vi = 1
vj = 200

contourf(V_axis[vi:vj], U_axis[ui:uj], (Φ_UV[ui:uj,vi:vj]),
    xlabel = "V",
    ylabel = "U",
    color=:thermal,
    wsize = (700,700),
    title = L"\Phi(U,V)",
    frame =true,
    legendfontsize=8,
    legend=:bottomleft)

p1 = plot!([U_axis[uslice]], seriestype="hline", label="V-V correlator", linewidth=1.5, color = "green",legend=:bottomleft)

# p2
i = 1
j = 200
contourf(V_axis[i:j], V_axis[i:j], real.((vv_crlt)[i:j,i:j]),
    title="V-V correlator @ U = $(U_axis[uslice])",
    xlabel = "V",
    ylabel = "V",
    color=:thermal,
    frame=true,
    legendfontsize=8,
legend=:bottomright
)

vi = 55
plot!([V_axis[vi]], seriestype="hline", label="V = $(V_axis[vi])", linewidth=1.5, color = "green")
p2 = plot!([V_axis[vi]], seriestype="vline", label="", linewidth=1.5, color = "green")

# p3
plot(V_axis[i:j],  real.(vv_crlt[i:j,vi]),
    xlabel = "V",
    frame=true,
    linewidth = 2, legendfontsize=8,
    label="V-V correlation",
xlim = (V_axis[i],V_axis[j]))

p3 = plot!([V_axis[vi]], seriestype="vline", label="V = $(V_axis[vi])", linewidth=1.5, color = "green", legend=:bottomleft)

plt = plot(p1, p2, p3, layout = grid(1, 3), size = (1200, 400))

#savefig(plt, out_dir*"/UU_crlt_NU$(NU)_vi$(vslice)_ui$(ui).pdf")

In [ ]:
uslice = 350
vv_crlt = h5read(your_dir*"/data_$(Nr)/VV_crlt_NV$(NV)_ui$(uslice).h5","crlt");

#p1
ui = 200
uj = 400
vi = 1
vj = 200

i = 1
j = 200

vslice = 55

###################################################################################
#p11
contourf(V_axis[vi:vj], U_axis[ui:uj], (Φ_UV[ui:uj,vi:vj]),
    xlabel = "V",
    ylabel = "U",
    color=:thermal,
    wsize = (700,700),
    title = L"\Phi(U,V)",
    frame =true,
    legendfontsize=8,
    legend=:bottomleft)

p11 = plot!([U_axis[uslice]], seriestype="hline", label="V-V correlator", linewidth=1.5, color = "green",legend=:bottomleft)

# p12
contourf(V_axis[i:j], V_axis[i:j], real.((vv_crlt)[i:j,i:j]),
    title="V-V correlator",
    xlabel = "V",
    ylabel = "V",
    color=:thermal,
    frame=true,
    legendfontsize=8,
legend=:bottomright
)

plot!([V_axis[vslice]], seriestype="hline", label="V = $(V_axis[vslice])", linewidth=1.5, color = "green")
p12 = plot!([V_axis[vslice]], seriestype="vline", label="", linewidth=1.5, color = "green")

# p13
plot(V_axis[i:j],  real.(vv_crlt[i:j,vslice]),
    xlabel = "V",
    frame=true,
    linewidth = 2, legendfontsize=8,
    label="",    
    title="V-V correlator",
    xlim = (V_axis[i],V_axis[j]))

p13 = plot!([V_axis[vi]], seriestype="vline", label="V = $(V_axis[vslice])", linewidth=1.5, color = "green", legend=:bottomleft)

###################################################################################
uslice = 320
vv_crlt = h5read(your_dir*"/data_$(Nr)/VV_crlt_NV$(NV)_ui$(uslice).h5","crlt");
#p21
contourf(V_axis[vi:vj], U_axis[ui:uj], (Φ_UV[ui:uj,vi:vj]),
    xlabel = "V",
    ylabel = "U",
    color=:thermal,
    wsize = (700,700),
    title = L"\Phi(U,V)",
    frame =true,
    legendfontsize=8,
    legend=:bottomleft)

p21 = plot!([U_axis[uslice]], seriestype="hline", label="V-V correlator", linewidth=1.5, color = "green",legend=:bottomleft)

# p22
contourf(V_axis[i:j], V_axis[i:j], real.((vv_crlt)[i:j,i:j]),
    title="V-V correlator",
    xlabel = "V",
    ylabel = "V",
    color=:thermal,
    frame=true,
    legendfontsize=8,
legend=:bottomright
)

plot!([V_axis[vslice]], seriestype="hline", label="V = $(V_axis[vslice])", linewidth=1.5, color = "green")
p22 = plot!([V_axis[vslice]], seriestype="vline", label="", linewidth=1.5, color = "green")

# p23
plot(V_axis[i:j],  real.(vv_crlt[i:j,vslice]),
    xlabel = "V",
    frame=true,
    linewidth = 2, legendfontsize=8,
    label="",    
    title="V-V correlator",
    xlim = (V_axis[i],V_axis[j]))

p23 = plot!([V_axis[vi]], seriestype="vline", label="V = $(V_axis[vslice])", linewidth=1.5, color = "green", legend=:bottomleft)

###################################################################################
uslice = 300
vv_crlt = h5read(your_dir*"/data_$(Nr)/VV_crlt_NV$(NV)_ui$(uslice).h5","crlt");
#p31
contourf(V_axis[vi:vj], U_axis[ui:uj], (Φ_UV[ui:uj,vi:vj]),
    xlabel = "V",
    ylabel = "U",
    color=:thermal,
    wsize = (700,700),
    title = L"\Phi(U,V)",
    frame =true,
    legendfontsize=8,
    legend=:bottomleft)

p31 = plot!([U_axis[uslice]], seriestype="hline", label="V-V correlator", linewidth=1.5, color = "green",legend=:bottomleft)

# p32
contourf(V_axis[i:j], V_axis[i:j], real.((vv_crlt)[i:j,i:j]),
    title="V-V correlator",
    xlabel = "V",
    ylabel = "V",
    color=:thermal,
    frame=true,
    legendfontsize=8,
legend=:bottomright
)

plot!([V_axis[vslice]], seriestype="hline", label="V = $(V_axis[vslice])", linewidth=1.5, color = "green")
p32 = plot!([V_axis[vslice]], seriestype="vline", label="", linewidth=1.5, color = "green")

# p33
plot(V_axis[i:j],  real.(vv_crlt[i:j,vslice]),
    xlabel = "V",
    frame=true,
    linewidth = 2, legendfontsize=8,
    label="",    
    title="V-V correlator",
    xlim = (V_axis[i],V_axis[j]))

p33 = plot!([V_axis[vi]], seriestype="vline", label="V = $(V_axis[vslice])", linewidth=1.5, color = "green", legend=:bottomleft)

###################################################################################
uslice = 280
vv_crlt = h5read(your_dir*"/data_$(Nr)/VV_crlt_NV$(NV)_ui$(uslice).h5","crlt");
#p41
contourf(V_axis[vi:vj], U_axis[ui:uj], (Φ_UV[ui:uj,vi:vj]),
    xlabel = "V",
    ylabel = "U",
    color=:thermal,
    wsize = (700,700),
    title = L"\Phi(U,V)",
    frame =true,
    legendfontsize=8,
    legend=:bottomleft)

p41 = plot!([U_axis[uslice]], seriestype="hline", label="V-V correlator", linewidth=1.5, color = "green",legend=:bottomleft)

# p42
contourf(V_axis[i:j], V_axis[i:j], real.((vv_crlt)[i:j,i:j]),
    title="V-V correlator",
    xlabel = "V",
    ylabel = "V",
    color=:thermal,
    frame=true,
    legendfontsize=8,
legend=:bottomright
)

plot!([V_axis[vslice]], seriestype="hline", label="V = $(V_axis[vslice])", linewidth=1.5, color = "green")
p42 = plot!([V_axis[vslice]], seriestype="vline", label="", linewidth=1.5, color = "green")

# p43
plot(V_axis[i:j],  real.(vv_crlt[i:j,vslice]),
    xlabel = "V",
    frame=true,
    linewidth = 2, legendfontsize=8,
    label="",    
    title="V-V correlator",
    xlim = (V_axis[i],V_axis[j]))

p43 = plot!([V_axis[vi]], seriestype="vline", label="V = $(V_axis[vslice])", linewidth=1.5, color = "green", legend=:bottomleft)


###################################################################################
plt = plot(
    p11, p12, p13, 
    p21, p22, p23, 
    p31, p32, p33, 
    p41, p42, p43, 
    layout = grid(4, 3), size = (1200, 4*400))

savefig(plt, out_dir*"/VV_crlt_NV$(NV)_uslices_vi$(vslice).pdf")